<a href="https://colab.research.google.com/github/Hasanen99/ML-Homeworks-AIDOJO/blob/main/Assignment_8_Hasanen_A_Sahib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 08: Advanced Sequence Applications

This Week's assignment is to train sequence models on the `Cafe Chatbot Data`.

Before starting copy this file and work on your own copy by following the below steps: <br>
`File > Save Copy in Drive`. Then add your name to the file (e.g., Assignment 07: Sequence Modeling - Zahraa Dhafer).

**DATASET**
This dataset consists of three csv files, you'll be working on the conversationo CSV file which contains two columns of questions and answers.<br><br>
**Submission Deadline: Saturday, 3/12/2022 at 3:00 PM**

**Requirements:**

1. Import all necessary libraries for the sequence modeling project.
2. Download the dataset (the link is provided below).
3. Read data from CSV file.
4. Prepare the data:
*   Clean the sentences by removing special characters.
*   Add a start and end token to each sentence.
*   Tokenize the sentences.
*   Pad each sentence to a maximum length.
<br>
5. Create the Data Pipeline for the Model
6. Create the Seq2Seq Model Architecture.<br>
**Note**: Use Adam optimizer and the appropriate loss function.
7. Create the Optimizer and the Loss Function.
8. Create the Training Step.
9. Create the Training Loop

**Note:** you can overfit your model.<br>
**HINTS:**
Set the new hyperparameters like vocabulary size, input length (i.e. max sequence length) in a separate cell after the import cell in your notebook (failing to do so will affect your style score)

**Note:** To get the best performance from the model, manually tune the hyperparameters of the model. 

Find relevant links below:<br>

[Assignment Colab File](https://colab.research.google.com/drive/1N6IcInQFF0iJdl8pruUvb4-rAacE0IBC?usp=sharing)<br>

[Dataset](https://www.kaggle.com/sonalibhoir/cafe-chatbot-dataset?select=conversationo.csv)

[Submission Form](https://docs.google.com/forms/d/e/1FAIpQLSf-WkcxjmZcxdUcu5OG7ZxFOsQu-qBy-u1UFNPRkWmjeW7w3g/viewform?usp=pp_url)<br>


Good luck and feel free to ask any questions in the or on the Questions channel.

## 1- Import all necessary libraries for the sequence modeling project

In [ ]:
# write your code below
import pandas as pd
import numpy as np 
import tensorflow as tf 
import re
from sklearn import model_selection

## 2- Download the dataset

In [ ]:
# write your code below
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d sonalibhoir/cafe-chatbot-dataset

mkdir: cannot create directory ‘/root/.kaggle’: File exists
  0% 0.00/7.55k [00:00<?, ?B/s]
100% 7.55k/7.55k [00:00<00:00, 7.88MB/s]


In [ ]:
! unzip /content/cafe-chatbot-dataset.zip

Archive:  /content/cafe-chatbot-dataset.zip
  inflating: Item_to_id.csv          
  inflating: conversationo.csv       
  inflating: food.csv                


## 3- Read data from CSV file

In [ ]:
data=pd.read_csv('/content/conversationo.csv')
data

,Question,answer
0,hey,Hello! How may I help you.
1,do u have coffee,"Yes sir Simple Coffee ,Cappuchino, Americano,..."
2,i will take one espresso and 5 americano,Sir thanks for your order. You have ordered 1 ...
3,anything special,"We have coffe,pastries,puff pastries and milks..."
4,suggest something,"We have coffe,pastries,puff pastries and milks..."
...,...,...
974,what is price of French Coffee,"Its our one of best, you can enjoy it at just ..."
975,what is price of Iced Coffee Late,"Its our one of best, you can enjoy it at just ..."
976,what is price of Latte Macchiato,"Its our one of best, you can enjoy it at just ..."
977,what is price of Wainans Choco Coffee,"Its our one of best, you can enjoy it at just ..."


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 979 entries, 0 to 978
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Question  961 non-null    object
 1   answer    939 non-null    object
dtypes: object(2)
memory usage: 15.4+ KB


In [ ]:
data.dropna(inplace=True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 937 entries, 0 to 978
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Question  937 non-null    object
 1   answer    937 non-null    object
dtypes: object(2)
memory usage: 22.0+ KB


After type error say:"'float' object has no attribute 'lower'", we should check the datatype of our data and fix them

In [ ]:
dtype_of_Q=set([type(i) for i in data['Question']])
dtype_of_A=set([type(i) for i in data['answer']])
print(f'Dtype of Questions: {dtype_of_Q}\nDtype of Answers: {dtype_of_A}')

Dtype of Questions: {<class 'str'>}
Dtype of Answers: {<class 'str'>}


**Note:** Problem solved after drop null lines, then all the data are string dtype

##4- Prepare the data:


### Clean the sentences by removing special characters

In [ ]:
# write your code below
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub(r'[^\w]',' ',text)
    text = re.sub('\w*\d\w*', '', text)
    text = " ".join(filter(lambda x:x[0]!="@", text.split()))
    return text

data['Question']=data['Question'].map(clean_text)
data['answer']=data['answer'].map(clean_text)
data

,Question,answer
0,hey,hello how may i help you
1,do u have coffee,yes sir simple coffee cappuchino americano au ...
2,i will take one espresso and americano,sir thanks for your order you have ordered esp...
3,anything special,we have coffe pastries puff pastries and milks...
4,suggest something,we have coffe pastries puff pastries and milks...
...,...,...
974,what is price of french coffee,its our one of best you can enjoy it at just rs
975,what is price of iced coffee late,its our one of best you can enjoy it at just rs
976,what is price of latte macchiato,its our one of best you can enjoy it at just rs
977,what is price of wainans choco coffee,its our one of best you can enjoy it at just rs


### Add a start and end token to each sentence

In [ ]:
# write your code below
def SE_adder(text):
  return f'<START> {text} <END>'

data['Question']=data['Question'].apply(SE_adder)
data['answer']=data['answer'].apply(SE_adder)

### Tokenize the sentences

In [ ]:
# write your code below
def tok(lang):
  tokenizer=tf.keras.preprocessing.text.Tokenizer(oov_token='<oov>',filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n',lower=False) #filters should not contain < and > symbols
  tokenizer.fit_on_texts(lang)

  output = tokenizer.texts_to_sequences(lang)
  output = tf.keras.preprocessing.sequence.pad_sequences(output, padding='post',truncating='post')
  return output,tokenizer

Ques_seq,Q_tok=tok(data['Question'])
Ans_seq,A_tok=tok(data['answer'])

In [ ]:
x_train,x_test,y_train,y_test=model_selection.train_test_split(Ques_seq,Ans_seq,test_size=0.1,random_state=42)

In [ ]:
x_train.shape[0]//32 + x_test.shape[0]//32

28

## 5- Create the Data Pipeline for the Model

In [ ]:
# write your code below
def pipline(x,y):
  d=tf.data.Dataset.from_tensor_slices((x,y))
  d=d.shuffle(1028)
  d=d.batch(32)
  d=d.prefetch(tf.data.AUTOTUNE)
  return d

train_dataset=pipline(x_train,y_train)
test_dataset=pipline(x_test,y_test)

In [ ]:
def decode_sentence(sequ,toke):
  textout=[]
  for t in sequ:
    if t!=0:
      textout.append(toke.index_word[t])
    else: pass
  textout=' '.join(textout)
  textout=textout.replace('<START> ','')
  textout=textout.replace(' <END>','')
  return textout

In [ ]:
for x,y in train_dataset.take(1):
  print(f'Q: {decode_sentence(x[0].numpy(),Q_tok)}\nA: {decode_sentence(y[0].numpy(),A_tok)}')

Q: do u have good pastery
A: yes sir


##6- Create the Seq2Seq Model Architecture

###Encoder Model

In [ ]:
in_vs=len(Q_tok.index_word)+1
out_vs=len(A_tok.index_word)+1
em_dim=200
B_size=32

In [ ]:
# write your code below
class Encoder(tf.keras.Model):
  def __init__(self,embedding_dim,batch_size,vocab_size):
    super(Encoder,self).__init__()
    self.batch_size=batch_size
    self.emb=tf.keras.layers.Embedding(input_dim=vocab_size,output_dim=embedding_dim)
    self.gru=tf.keras.layers.GRU(256,return_sequences=True,return_state=True)
    
  def call(self,x,hidden_state):
    x=self.emb(x)
    out,state=self.gru(x,initial_state=hidden_state)    
    return out,state

  def encoder_initializer(self):
    return tf.zeros((self.batch_size, 256)) 

### Decoder Model

In [ ]:
# write your code below
class Decoder(tf.keras.Model):
  def __init__(self,embedding_dim,batch_size,vocab_size,units):
    super(Decoder,self).__init__()
    self.batch_size=batch_size
    self.units=units
    self.emb=tf.keras.layers.Embedding(input_dim=vocab_size,output_dim=embedding_dim)
    self.gru=tf.keras.layers.GRU(256,return_sequences=True,return_state=True)
    
    self.dense=tf.keras.layers.Dense(units=self.units)
  def call(self,x,hidden_state):
    x=self.emb(x)
    out,state=self.gru(x,initial_state=hidden_state)
    out = tf.reshape(out, (-1, out.shape[2]))
    out = tf.nn.softmax(self.dense(out))
    return out,state

In [ ]:
my_enc=Encoder(embedding_dim=em_dim,batch_size=B_size,vocab_size=in_vs)
my_dec=Decoder(embedding_dim=em_dim,batch_size=B_size,vocab_size=out_vs,units=out_vs)

## 7- Create the Optimizer and the Loss Function

In [ ]:
# write your code below
OPT=tf.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  Loss = loss_object(real, pred)
  mask = tf.cast(mask, dtype=Loss.dtype)
  loss = Loss * mask
  return tf.reduce_mean(Loss)

##8- Create the Training Step

In [ ]:
train_loss = tf.metrics.Mean(name='train loss')
test_loss = tf.metrics.Mean(name='test loss')

In [ ]:
# write your code below

@tf.function
def train_step(input, target, encoder_hidden_state):

  loss = 0
  with tf.GradientTape() as tape:
    encoder_output, encoder_hidden_state = my_enc(input, encoder_hidden_state)
    decoder_hidden_state = encoder_hidden_state
    decoder_input = tf.expand_dims([A_tok.word_index['<START>']] * input.shape[0], 1)
    for t in range(1, target.shape[1]):
      prediction, decoder_hidden_state = my_dec(decoder_input, decoder_hidden_state)
      loss += loss_function(target[:, t], prediction)
      decoder_input = tf.expand_dims(target[:, t], 1)
  batch_loss = loss / int(target.shape[1]) 
  variables = my_enc.trainable_variables + my_dec.trainable_variables
  gradients = tape.gradient(loss, variables)
  OPT.apply_gradients(zip(gradients, variables))
  train_loss(batch_loss)
  return batch_loss  

## 9- Create the Testing Loop

In [ ]:
# write your code below
@tf.function
def test_step(input, target, encoder_hidden_state):

  loss = 0
  encoder_output, encoder_hidden_state = my_enc(input, encoder_hidden_state)
  decoder_hidden_state = encoder_hidden_state
  decoder_input = tf.expand_dims([A_tok.word_index['<START>']] * input.shape[0], 1) 
  for t in range(1, target.shape[1]):
    prediction, decoder_hidden_state = my_dec(decoder_input, decoder_hidden_state)
    loss += loss_function(target[:, t], prediction)
    # teacher forcing
    decoder_input = tf.expand_dims(target[:, t], 1)
  batch_loss = loss / int(target.shape[1]) 
  test_loss(batch_loss)

Calculate number of target of keras progbar, and each part of train half and test half

In [ ]:
Train_take = x_train.shape[0]//32
Test_take = x_test.shape[0]//32
All_take = Train_take + Test_take

In [ ]:
Epoch = 50

for epoch in range(Epoch):
  train_loss.reset_states()
  test_loss.reset_states()
  encoder_hidden_state = my_enc.encoder_initializer()
  progress_bar = tf.keras.utils.Progbar(target= All_take)
  batch_count = 0
  for (batch, (input, target)) in enumerate(train_dataset.take(Train_take)):
    batch_count += 1
    train_step(input, target, encoder_hidden_state)
    progress_bar.update(batch_count)

  for (batch, (input, target)) in enumerate(test_dataset.take(Test_take)):
    batch_count += 1
    test_step(input, target, encoder_hidden_state)
    progress_bar.update(batch_count)

  print(f'Epoch: {epoch + 1} -  loss: {train_loss.result()} -------- val_loss: {test_loss.result()}')

28/28 [==============================] - 100s 922ms/step
Epoch: 1 -  loss: 2.5038657188415527 -------- val_loss: 1.5732766389846802
28/28 [==============================] - 6s 202ms/step
Epoch: 2 -  loss: 1.1397812366485596 -------- val_loss: 1.2616807222366333
28/28 [==============================] - 6s 201ms/step
Epoch: 3 -  loss: 1.0367209911346436 -------- val_loss: 1.1724615097045898
28/28 [==============================] - 6s 200ms/step
Epoch: 4 -  loss: 0.9398015141487122 -------- val_loss: 1.1464905738830566
28/28 [==============================] - 5s 199ms/step
Epoch: 5 -  loss: 0.7724625468254089 -------- val_loss: 0.9182495474815369
28/28 [==============================] - 6s 200ms/step
Epoch: 6 -  loss: 0.6759061217308044 -------- val_loss: 0.8503658771514893
28/28 [==============================] - 6s 201ms/step
Epoch: 7 -  loss: 0.5872515439987183 -------- val_loss: 0.8456438779830933
28/28 [==============================] - 5s 198ms/step
Epoch: 8 -  loss: 0.5019118189811

predicting function

In [ ]:
# create the chatbot function
# the chatbot function takes in the question as input and answers the input sentence 
def chatbot(sentence):
  
  # clean the input question sentence 
  sentence = clean_text(sentence)
  # add the start token to the sentence
  sentence =SE_adder(sentence)
  # tokenize the sentence
  inputs = Q_tok.texts_to_sequences([sentence])
  # pad the sentence
  inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs,padding='post',truncating='post')
  
  # initalize the hidden state of the encoder to zeros
  hidden = [tf.zeros((1, 256))]
  # pass the sentence to the encoder with the hidden state as the initial hidden state
  enc_out, enc_hidden = my_enc(inputs, hidden)
  # set the initial decoder hidden state to the encoder hidden state
  dec_hidden = enc_hidden
  # create the start token
  # start_token shape == (batch_size, 1)
  # repeat the start token for the batch size times
  dec_input = tf.expand_dims([A_tok.word_index['<START>']], 0)
  # create the result string
  result = ''
  # loop over the length of the sentence (32)

  for t in range(32):
    # passing the encoder output and the decoder hidden state to the decoder make sure the decoder input is the previous predicted word
    predictions, dec_hidden = my_dec(dec_input, dec_hidden)

    # getting the predicted word index
    predicted_id = tf.argmax(predictions[0]).numpy()
    # getting the predicted word using the predicted index
    # add the predicted word to the result string 
    result += A_tok.index_word[predicted_id] + ' '
    # if the predicted word is the <end> token then stop the loop
    if A_tok.index_word[predicted_id] == '<END>':
      # remove the <start> and <END> tokens from the result string
      result = result.replace('<START> ', '')
      result = result.replace(' <END> ','')
      # remove the <START> and <END> tokens from the sentence string
      sentence = sentence.replace('<START> ', '')
      sentence = sentence.replace(' <END>', '')
      return  sentence, result
    # using the predicted word as the next decoder input
    dec_input = tf.expand_dims([predicted_id], 0)

  # remove the <START> and <END> tokens from the result string
  result = result.replace('<START> ', '')
  result = result.replace('<END>','')
  # remove the <START> and <END> tokens from the sentence string
  sentence = sentence.replace('<START> ', '')
  sentence = sentence.replace('<END>', '')
  
  # return the result string and the original sentence
  return sentence, result

In [ ]:
chatbot('price of french coffee')

('price of french coffee', 'its our one of best you can enjoy it at just rs')